#Setup API kaggle acount

In [2]:
import os
os.listdir()

['.config', 'kaggle.json', 'sample_data']

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Download the dataset

In [9]:
!pip install kagglehub

In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessemostipak/hotel-booking-demand")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'hotel-booking-demand' dataset.
Path to dataset files: /kaggle/input/hotel-booking-demand


In [11]:
import os
os.listdir(path)

['hotel_bookings.csv']

In [12]:
import pandas as pd

csv_path = os.path.join(path, "hotel_bookings.csv")
df = pd.read_csv(csv_path)

df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


#Pra-process on sql based

In [15]:
import sqlite3

conn = sqlite3.connect("hotel_bookings.db")
df.to_sql("bookings", conn, if_exists="replace", index=False)

119390

In [16]:
pd.read_sql_query("SELECT COUNT(*) FROM bookings;", conn)

,COUNT(*)
0,119390


Scheme check

In [17]:
pd.read_sql_query("PRAGMA table_info(bookings);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,hotel,TEXT,0,None,0
1,1,is_canceled,INTEGER,0,None,0
2,2,lead_time,INTEGER,0,None,0
3,3,arrival_date_year,INTEGER,0,None,0
4,4,arrival_date_month,TEXT,0,None,0
5,5,arrival_date_week_number,INTEGER,0,None,0
6,6,arrival_date_day_of_month,INTEGER,0,None,0
7,7,stays_in_weekend_nights,INTEGER,0,None,0
8,8,stays_in_week_nights,INTEGER,0,None,0
9,9,adults,INTEGER,0,None,0


use only needed feature

In [25]:
query = """
SELECT
  hotel,
  lead_time,
  adr,
  market_segment,
  customer_type,
  deposit_type,
  previous_cancellations,
  is_canceled
FROM bookings
WHERE adr > 0
  AND lead_time >= 0;
"""
pd.read_sql_query(query, conn).head()


,hotel,lead_time,adr,market_segment,customer_type,deposit_type,previous_cancellations,is_canceled
0,Resort Hotel,7,75.0,Direct,Transient,No Deposit,0,0
1,Resort Hotel,13,75.0,Corporate,Transient,No Deposit,0,0
2,Resort Hotel,14,98.0,Online TA,Transient,No Deposit,0,0
3,Resort Hotel,14,98.0,Online TA,Transient,No Deposit,0,0
4,Resort Hotel,0,107.0,Direct,Transient,No Deposit,0,0


on SQL it will be:
SELECT
  hotel,
  lead_time,
  adr,
  market_segment,
  customer_type,
  deposit_type,
  previous_cancellations,
  is_canceled
FROM bookings
WHERE adr > 0
  AND lead_time >= 0;

In [26]:
pd.read_sql_query("""
SELECT
  market_segment,
  COUNT(*) AS total_bookings,
  AVG(is_canceled) AS cancellation_rate
FROM bookings
GROUP BY market_segment
ORDER BY cancellation_rate DESC;
""", conn)

,market_segment,total_bookings,cancellation_rate
0,Undefined,2,1.000000
1,Groups,19811,0.610620
2,Online TA,56477,0.367211
3,Offline TA/TO,24219,0.343160
4,Aviation,237,0.219409
5,Corporate,5295,0.187347
6,Direct,12606,0.153419
7,Complementary,743,0.130552


on SQL it will be:
SELECT
  market_segment,
  COUNT(*) AS total_bookings,
  AVG(is_canceled) AS cancellation_rate
FROM bookings
GROUP BY market_segment
ORDER BY cancellation_rate DESC;


Light SQL feature engineering

In [27]:
query = """
SELECT
  hotel,
  market_segment,
  customer_type,
  deposit_type,

  CASE
    WHEN lead_time <= 7 THEN 'short'
    WHEN lead_time <= 30 THEN 'medium'
    ELSE 'long'
  END AS lead_time_group,

  CASE
    WHEN adr >= 200 THEN 1
    ELSE 0
  END AS high_adr,

  CASE
    WHEN previous_cancellations > 0 THEN 1
    ELSE 0
  END AS has_previous_cancellation,

  is_canceled
FROM bookings
WHERE adr > 0
  AND lead_time >= 0;
"""
df_fe = pd.read_sql_query(query, conn)
df_fe.head()

,hotel,market_segment,customer_type,deposit_type,lead_time_group,high_adr,has_previous_cancellation,is_canceled
0,Resort Hotel,Direct,Transient,No Deposit,short,0,0,0
1,Resort Hotel,Corporate,Transient,No Deposit,medium,0,0,0
2,Resort Hotel,Online TA,Transient,No Deposit,medium,0,0,0
3,Resort Hotel,Online TA,Transient,No Deposit,medium,0,0,0
4,Resort Hotel,Direct,Transient,No Deposit,short,0,0,0


with;
CASE(1)=Category based on booking days to d-day
CASE(2)=Category based on price
CASE(3)=Category based on previous cancellation

#Encoding

In [28]:
categorical_cols = [
    "hotel",
    "meal",
    "market_segment",
    "distribution_channel",
    "customer_type",
    "deposit_type"
]

df_encoded = pd.get_dummies(
    df,
    columns=categorical_cols,
    drop_first=True
)

In [29]:
print(df.shape)
print(df_encoded.shape)

(119390, 32)
(119390, 47)


NLP

In [30]:
df_encoded["room_changed"] = (
    df["reserved_room_type"] != df["assigned_room_type"]
).astype(int)

Room encoding

In [31]:
country_freq = df["country"].value_counts(normalize=True)
df_encoded["country_freq"] = df["country"].map(country_freq)

Country encoding

In [32]:
df_encoded = df_encoded.drop(
    columns=["reserved_room_type", "assigned_room_type", "country"]
)

In [33]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 46 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   is_canceled                     119390 non-null  int64  
 1   lead_time                       119390 non-null  int64  
 2   arrival_date_year               119390 non-null  int64  
 3   arrival_date_month              119390 non-null  object 
 4   arrival_date_week_number        119390 non-null  int64  
 5   arrival_date_day_of_month       119390 non-null  int64  
 6   stays_in_weekend_nights         119390 non-null  int64  
 7   stays_in_week_nights            119390 non-null  int64  
 8   adults                          119390 non-null  int64  
 9   children                        119386 non-null  float64
 10  babies                          119390 non-null  int64  
 11  is_repeated_guest               119390 non-null  int64  
 12  previous_cancell

#FEATURE IMPACT

Cancellation ratio

In [34]:
df["is_canceled"].value_counts(normalize=True)

,proportion
is_canceled,
0,0.629584
1,0.370416


NUMERIC FEATURES — GROUP COMPARISON

In [35]:
df.groupby("is_canceled")["lead_time"].mean()

,lead_time
is_canceled,
0,79.984687
1,144.848815


In [36]:
df.groupby("is_canceled")["lead_time"].median()

,lead_time
is_canceled,
0,45.0
1,113.0


for main features

In [37]:
numeric_features = [
    "lead_time",
    "adr",
    "previous_cancellations",
    "booking_changes",
    "days_in_waiting_list"
]

df.groupby("is_canceled")[numeric_features].mean()

,lead_time,adr,previous_cancellations,booking_changes,days_in_waiting_list
is_canceled,,,,,
0,79.984687,99.987693,0.015792,0.293364,1.589868
1,144.848815,104.964333,0.208348,0.098340,3.564083


CATEGORICAL FEATURES — CANCEL RATE

In [38]:
df.groupby("deposit_type")["is_canceled"].mean().sort_values(ascending=False)

,is_canceled
deposit_type,
Non Refund,0.993624
No Deposit,0.283770
Refundable,0.222222


In [40]:
categorical_features = [
    "market_segment",
    "customer_type",
    "hotel",
    "room_changed"
]

for col in categorical_features:
    print(f"\n{col.upper()}")
    if col == "room_changed":
        # 'room_changed' was added to df_encoded
        print(df_encoded.groupby(col)["is_canceled"].mean().sort_values(ascending=False).head())
    else:
        # Other categorical features are in the original df
        print(df.groupby(col)["is_canceled"].mean().sort_values(ascending=False).head())


MARKET_SEGMENT
market_segment
Undefined        1.000000
Groups           0.610620
Online TA        0.367211
Offline TA/TO    0.343160
Aviation         0.219409
Name: is_canceled, dtype: float64

CUSTOMER_TYPE
customer_type
Transient          0.407463
Contract           0.309617
Transient-Party    0.254299
Group              0.102253
Name: is_canceled, dtype: float64

HOTEL
hotel
City Hotel      0.417270
Resort Hotel    0.277634
Name: is_canceled, dtype: float64

ROOM_CHANGED
room_changed
0    0.415629
1    0.053764
Name: is_canceled, dtype: float64


Country based

In [41]:
country_freq = df["country"].value_counts(normalize=True)
df["country_freq"] = df["country"].map(country_freq)

In [42]:
df.groupby("is_canceled")["country_freq"].mean()

,country_freq
is_canceled,
0,0.154094
1,0.272417


Ranks of most probable feature

In [43]:
impact = {}

for col in numeric_features:
    impact[col] = abs(
        df[df["is_canceled"] == 1][col].mean() -
        df[df["is_canceled"] == 0][col].mean()
    )

pd.Series(impact).sort_values(ascending=False)

,0
lead_time,64.864128
adr,4.976640
days_in_waiting_list,1.974215
booking_changes,0.195024
previous_cancellations,0.192557


"Exploratory analysis shows that booking cancellation is strongly associated with longer lead time, higher previous cancellation history, and non-refundable deposit types. Certain market segments also exhibit significantly higher cancellation rates, indicating behavioral patterns rather than random noise."